# core

> The core classes and functionality for SentenceGraph.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from typing import List
from enum import Enum
import pandas as pd
from collections import namedtuple
import numpy as np
import random as random
import torch

In [ ]:
#| export
class Format(Enum):
  Python = 0
  Torch = 1
  Numpy = 2
  Pandas = 3

In [ ]:
#| export
TextNode = namedtuple('TextNodeTuple', 'id text')

In [ ]:
#| export
class SentenceGraph:
    def __init__(self, model_name: str = None, model: None = None):
        if model_name == None or model_name == "" or model == None:
            self.model = model = SentenceTransformer('all-MiniLM-L6-v2')
        elif model is not None:
            self.model = model
        elif model_name is not None:
          self.model = model = SentenceTransformer(model_name)

    def create_graph(self, sentences: List[TextNode], format: Format = Format.Python) -> List[List[float]]:
        # TODO: Wrap this all in a function to export a functional version of this as well.
        sentence_embeddings = self.model.encode([node.text for node in sentences])

        graph_len = len(sentence_embeddings)
        similarity_graph = [[0 for x in range(graph_len)] for y in range(graph_len)]

        for i, embeddingA in enumerate(sentence_embeddings):
          for j, embeddingB in enumerate(sentence_embeddings):
            similarity = cos_sim(embeddingA, embeddingB)

            similarity = similarity.tolist()[0][0]

            similarity_graph[i][j] = similarity
        
        if format == Format.Pandas:
          index = [node.id for node in sentences]
          df = pd.DataFrame(similarity_graph, index = index,
                                          columns = index)
          return df
        elif format == Format.Numpy:
          return np.array(similarity_graph)
        elif format == Format.Torch:
          return torch.asarray(similarity_graph)
        else:
          return similarity_graph

    def _select_top_edges(self, sim_matrix, extract_num, beta, lambda1, lambda2) -> List[int]:
        """ 
          Code taken from: https://github.com/mswellhao/PacSum
          Given a similiarity matrix of text edge scores. Select the most semantically relevant edges.
        """
        # Compute the edge threshold and new edge 
        min_score = sim_matrix.min()
        max_score = sim_matrix.max()
        edge_threshold = min_score + beta * (max_score - min_score)
        new_edge_scores = sim_matrix - edge_threshold
        forward_scores, backward_scores, _ = self._compute_new_edge_scores(new_edge_scores)
        forward_scores = 0 - forward_scores

        paired_scores = []
        for node in range(len(forward_scores)):
            paired_scores.append([node,  lambda1 * forward_scores[node] + lambda2 * backward_scores[node]])

        #shuffle to avoid any possible bias
        random.shuffle(paired_scores)
        paired_scores.sort(key = lambda x: x[1], reverse = True)
        extracted = [item[0] for item in paired_scores[:extract_num]]

        return extracted

    def _compute_new_edge_scores(self, similarity_matrix, edge_threshold = 0):
        """ 
          Code taken from https://github.com/mswellhao/PacSum 
          Grabs the most important edges based off of an edge threshold
        """
        forward_scores = [0 for i in range(len(similarity_matrix))]
        backward_scores = [0 for i in range(len(similarity_matrix))]
        edges = []
        for i in range(len(similarity_matrix)):
            for j in range(i+1, len(similarity_matrix[i])):
                edge_score = similarity_matrix[i][j]
            if edge_score > edge_threshold:
                forward_scores[j] += edge_score
                backward_scores[i] += edge_score
                edges.append((i,j,edge_score))

        return np.asarray(forward_scores), np.asarray(backward_scores), edges

    def extract_summary(self, textNodes: List[TextNode], num_nodes = 3, beta = 3, lambda1 = -0.2, lambda2 = -0.2) -> str:
        """ 
          Given a list of text, create an extractive summary with the top n nodes.
          Beta, lambda1, and lambda2 come from the https://github.com/mswellhao/PacSum paper along with the current extractor algorithim.
        """
        sim_matrix = self.create_graph(textNodes, format=Format.Numpy)

        top_sentences = self._select_top_edges(sim_matrix, num_nodes, beta, lambda1, lambda2)
        summary = [textNodes[idx].text for idx in top_sentences]
        return ' '.join(summary)


NameError: name 'Format' is not defined

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()